In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import time

import cv2
import torch

from nanodet.data.transform import Pipeline
from nanodet.model.arch import build_model
from nanodet.util import load_model_weight

In [3]:
import os

import onnx
import onnxsim
import torch

from nanodet.model.arch import build_model
from nanodet.util import Logger, cfg, load_config, load_model_weight

Installing onnxruntime by `/home/ec2-user/anaconda3/envs/pytorch_p39/bin/python -m pip install onnxruntime`, please
wait for a moment..

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 24.0 MB/s eta 0:00:00


In [4]:
cfg_path = "config/nanoinstance-mask-512.yml"
model_path = "workspace/nanodet-plus-cucumber_512_test/model_last.ckpt"
out_path = "test.onnx"
input_shape = (512,512)
load_config(cfg, cfg_path)

In [8]:
dir(logger)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'log',
 'log_dir',
 'rank',
 'scalar_summary']

In [9]:
logger = Logger(-1, cfg.save_dir, False)
model = build_model(cfg.model)
checkpoint = torch.load(model_path, map_location=lambda storage, loc: storage)
logger.log("Loading trained model weights!")
load_model_weight(model, checkpoint, logger)

model size is  1.0x
init weights...
=> loading pretrained model https://download.pytorch.org/models/shufflenetv2_x1-5666bf0f80.pth


[root][05-26 08:11:37]INFO:Loading trained model weights!
[root][05-26 08:11:37]INFO:Loading trained model weights!
[root][05-26 08:11:37]INFO:Loading trained model weights!
[root][05-26 08:11:37]INFO:Loading trained model weights!


In [10]:
pipeline = Pipeline(cfg.data.val.pipeline, cfg.data.val.keep_ratio)

In [11]:
cfg.data.val.input_size

[512, 512]

In [12]:
img=cv2.imread("data/cucumbers/0.png")
height, width = img.shape[:2]
img_info = {}
img_info["height"] = height
img_info["width"] = width
meta = dict(img_info=img_info, raw_img=img, img=img)

In [13]:
meta = pipeline(meta,cfg.data.val.input_size)

In [14]:
img=torch.from_numpy(meta["img"].transpose(2, 0, 1)).unsqueeze(0)

In [15]:
img.shape

torch.Size([1, 3, 512, 512])

In [16]:
dummy_input = img

In [17]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

In [18]:
torch.onnx.export(
    model,
    dummy_input,
    out_path,
    export_params=True, 
    verbose=False,
    keep_initializers_as_inputs=True,
    opset_version=11,
    input_names=["data"],
    output_names=["output"],
)

In [19]:
logger.log("finished exporting onnx ")

[root][05-26 08:13:30]INFO:finished exporting onnx 
[root][05-26 08:13:30]INFO:finished exporting onnx 
[root][05-26 08:13:30]INFO:finished exporting onnx 
[root][05-26 08:13:30]INFO:finished exporting onnx 


In [20]:
out=model(dummy_input)

In [ ]:
out[0].shape, out[1][0].shape,out[1][2].shape

In [ ]:
import onnxruntime
ort_session = onnxruntime.InferenceSession("test.onnx")